## Data_Load & Split

In [1]:
# 구글 드라이브 입력
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import pandas as pd
import numpy as np
import os
import warnings


# 파일 위치 고정
os.chdir("/content/gdrive/MyDrive/신용카드 사기 데이콘")


trn = pd.read_csv('open/train.csv').drop(columns=['ID'])
val_ = pd.read_csv('open/val.csv').drop(columns=['ID'])
val = val_.iloc[:,:-1]
lab = val_.iloc[:,-1]

tst = pd.read_csv('open/test.csv')
tst_id = tst[['ID']]
tst = tst.drop(columns=['ID'])

warnings.filterwarnings(action='ignore')

from sklearn.preprocessing import StandardScaler
std = StandardScaler()
trn.iloc[:,:] = std.fit_transform(trn)
val.iloc[:,:] = std.fit_transform(val)
tst.iloc[:,:] = std.fit_transform(tst)

## baseline 참고
contamination = 0.0010551491277433877

nor = val_[val_['Class']==0]
abn = val_[val_['Class']==1]
X_trn = nor.iloc[:26000,:-1]
X_tst = pd.concat([nor.iloc[26000:,:-1], abn.iloc[:,:-1]])
y_tst = pd.concat([nor.iloc[26000:,-1],abn.iloc[:,-1]])

## Pyod Anomaly Detection package

In [3]:
!pip install pyod
# git clone https://github.com/yzhao062/pyod.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.7-py3-none-any.whl size=181101 sha256=d18f13603fc184a9121a40e6dbff5b387c6372c9313047e3af108adeb142bab1
  Stored in directory: /root/.cache/pip/wheels/f7/e2/c1/1c7fd8b261e72411f6509afb429c84532e40ddcd96074473f4
Successfully built pyod


In [4]:
! pip install suod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for suod: filename=suod-0.0.8-py3-none-any.whl size=2155017 sha256=24e728ca8abbae172d17dad2e57777993d79ab1ca97237f4ab0245c6dc725b6a
  Stored in directory: /root/.cache/pip/wheels/14/d0/a2/ffac7a9e056dc6644a525bcef6a4ed08c8b0ee1a45a96224c8
  Created wheel for combo: filename=combo-0.1.3-py3-none-any.whl size=42885 sha256=228a28f74eed65cd5ab4d9a05fa090b8320eb3081d7e507cb5a3a52b9c67ec51
  Stored in directory: /root/.cache/pip/wheels/ec/44/39/0667fea44a2dfe692cc2a51f0f79ea49b9dee7def53594ef2e
Successfully built suod combo


In [5]:
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, f1_score
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.suod import SUOD

ecod = ECOD(contamination=contamination)
copod = COPOD(contamination=contamination)
suod = SUOD(contamination=contamination)

ecod.fit(trn)
copod.fit(trn)
suod.fit(trn)

ecod_pred = ecod.predict(val)
copod_pred = copod.predict(val)
suod_pred = suod.predict(val)

print('='*30)
print('ecod_validation_confusion_matrix\n',confusion_matrix(lab, ecod_pred,labels=[1,0]))
print('ecod_validation_f1_score\n', f1_score(lab, ecod_pred,labels=[1,0]))
print('='*30)
print('copod_validation_confusion_matrix\n',confusion_matrix(lab, copod_pred,labels=[1,0]))
print('copod_validation_f1_score\n', f1_score(lab, copod_pred,labels=[1,0]))
print('='*30)
print('suod_validation_confusion_matrix\n',confusion_matrix(lab, suod_pred,labels=[1,0]))
print('suod_validation_f1_score\n', f1_score(lab, suod_pred,labels=[1,0]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.2min finished


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ecod_validation_confusion_matrix
 [[   13    17]
 [   20 28412]]
ecod_validation_f1_score
 0.4126984126984127
copod_validation_confusion_matrix
 [[   13    17]
 [   14 28418]]
copod_validation_f1_score
 0.456140350877193
suod_validation_confusion_matrix
 [[   12    18]
 [   16 28416]]
suod_validation_f1_score
 0.4137931034482759


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.9s finished


In [6]:
# suod : Anomaly detection ensemble model
suod.get_params()

{'approx_clf': None,
 'approx_clf_list': None,
 'approx_flag_global': True,
 'approx_ng_clf_list': None,
 'base_estimators': [LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
    metric_params=None, n_jobs=1, n_neighbors=15, novelty=True, p=2),
  LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
    metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2),
  HBOS(alpha=0.1, contamination=0.1, n_bins=10, tol=0.5),
  HBOS(alpha=0.1, contamination=0.1, n_bins=20, tol=0.5),
  COPOD(contamination=0.1, n_jobs=1),
  IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
      max_samples='auto', n_estimators=50, n_jobs=1, random_state=None,
      verbose=0),
  IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
      max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
      verbose=0),
  IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    

In [7]:
from sklearn.metrics import confusion_matrix, f1_score
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE

In [8]:
ae = AutoEncoder(
    hidden_neurons=[128,64,32,16,16,32,64,128],
    validation_size=0.2,
    contamination=contamination,
    verbose=1,
    epochs=15
)
vae = VAE(
    encoder_neurons=[128,64,32,16],
    decoder_neurons=[16,32,64,128],
    validation_size=0.2,
    contamination=contamination,
    verbose=1,
    epochs=15
)

ae.fit(trn)
vae.fit(trn)

ae_pred = ae.predict(val)
vae_pred = vae.predict(val)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                930       
                                                                 
 dropout (Dropout)           (None, 30)                0         
                                                                 
 dense_1 (Dense)             (None, 30)                930       
                                                                 
 dropout_1 (Dropout)         (None, 30)                0         
                                                                 
 dense_2 (Dense)             (None, 128)               3968      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8

In [9]:
print('='*30)
print('ae_validation_confusion_matrix\n',confusion_matrix(lab, ae_pred,labels=[1,0]))
print('ae_validation_f1_score\n', f1_score(lab, ae_pred,labels=[1,0]))
print('='*30)
print('vae_validation_confusion_matrix\n',confusion_matrix(lab, vae_pred,labels=[1,0]))
print('vae_validation_f1_score\n', f1_score(lab, vae_pred,labels=[1,0]))

ae_validation_confusion_matrix
 [[   11    19]
 [   21 28411]]
ae_validation_f1_score
 0.3548387096774193
vae_validation_confusion_matrix
 [[   11    19]
 [   21 28411]]
vae_validation_f1_score
 0.3548387096774193
